# 1.Call libraries, functions, and clean data initially

In [12]:
# Loading required libraries
pacman::p_load(tidyverse, rjson, DEoptim, doParallel, readr, here, jsonlite, modi, gplots)
knitr::opts_chunk$set(echo = TRUE)

source("functions/functions_behavioral_action_analysis.R")
source("functions/functions_data_modification_action_analysis.R")
#source("functions_behavioral_action_analysis.R")
#source("functions_behavior_phases.R")

library(tidyverse)
library(readr)
library(dplyr)

raw_data <- read_csv("data/data_social_all_participants.csv")
data <- select_important_variables(raw_data)

#Now, we are going to check if the data
miss <- check_for_clean(data)
data <- clean_data(data)

trim_data <- process_data(data, trim_beyond_gem = TRUE)
ntrim_data <- process_data(data, trim_beyond_gem = FALSE)
summary_data <- create_summary_data(ntrim_data)
ntrim_data <- determine_social_information(ntrim_data)


Rows: 6000 Columns: 24
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): group, social_info_use, performance_group_f, demo_type
dbl (20): points, cells, social_info, unique_rounds, env_number, gender, soc...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "75 rows excluded. Call 'miss' dataframe to check."


# 2. Determine descriptive statistics about data set

In [13]:
## Nothing here yet
descriptive_data <- raw_data %>%
  select(uniqueID, gender, group, tot_points) %>%
  distinct(uniqueID, .keep_all = TRUE)

table(subset(descriptive_data, group == "adolescents", select = gender)$gender)

nrow(subset(descriptive_data,group == "adults"))
nrow(subset(descriptive_data,group == "adolescents"))


 1 
10 

[1] 0

[1] 10

# 3. Create action_data set for the action analyses

## a. First overall transformation from data to action_data and summary_data with social information

In [14]:
# We dedicate a unique data set to action and transform data from there.

action_data <- process_data(data, trim_beyond_gem = FALSE)
action_summary_data <- create_summary_data(action_data)

action_data <- determine_social_information(action_data)
action_summary_data <- add_social_value_variables(action_data, action_summary_data)

action_data <- create_action_data(action_data)

#Was determined in the actual analyses
action_data <- add_social_cluster(action_data)
player_summary_data_gem_found <- create_player_summary(action_summary_data,1)
player_summary_data_gem_not_found <- create_player_summary(action_summary_data,0)
player_summary_data_full <- create_player_summary(action_summary_data,2)

## b. Specific datasets for each of the three actions

In [9]:
###i. Personal Exploration

pers_explore_data <- action_data %>% filter(social_info_use == "ignore", known_cell == FALSE)

#Either excluding or including missmatches and missing values. I decided to include them, except for checking effects of exclusion.

pers_explore_data <- pers_explore_data[!is.na(pers_explore_data$previous_points) & !is.na(pers_explore_data$points),]

player_pers_explore_summary <- create_player_pers_explore_summary(pers_explore_data)

average_pers_explore_summary <- create_average_pers_explore_summary(pers_explore_data)

trial_pers_explore_summary <- create_trial_pers_explore_summary(pers_explore_data)


###ii. Social Exploration

soc_explore_data <- action_data %>% filter(social_info_use == "copy", known_cell == FALSE, social_value>0)
soc_copy_data <- action_data %>% filter(social_info_use == "copy", gem_found == 0)

#Either excluding or including missmatches and missing values. I decided to include them, except for checking effects of exclusion.

soc_explore_data <- soc_explore_data[!is.na(soc_explore_data$previous_points) & !is.na(soc_explore_data$points),]

player_soc_explore_summary <- create_player_soc_explore_summary(soc_explore_data)

average_soc_explore_summary <- create_average_soc_explore_summary(soc_explore_data)

trial_soc_explore_summary <- create_trial_soc_explore_summary(soc_explore_data)


###iii. Repetition Action

repeat_data <- create_repeat_data(action_data)

repeat_summary <- create_repeat_summary(repeat_data)

average_repeat_summary <- create_average_measures_repeated(repeat_data)

`summarise()` has grouped output by 'player'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'player'. You can override using the
`.groups` argument.
